In [ ]:
#install transformers library
!pip install transformers -U -q

# install sentencepiece library
!pip install sentencepiece

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers.optimization import AdamW
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


print('Model loading started')
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="en_XX", tgt_lang="ta_IN")
print('Model loading done')






Model loading started


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model loading done


In [ ]:


#src_text = "state with the second"
#tgt_text = "இரண்டாவது மாநிலம்  "

#src_text = "to Tamil Nadu"
#tgt_text = "தமிழ்நாட்டிடம் "


src_text = "in India"
tgt_text = "இந்தியாவின்   "




model_inputs = tokenizer(src_text, return_tensors="pt")
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_text, return_tensors="pt").input_ids

# Set up the optimizer and training settings
optimizer = AdamW(model.parameters(), lr=1e-4)
model.train()

print('Fine-tuning started')
for i in range(1):
 optimizer.zero_grad()
 output = model(**model_inputs, labels=labels) # forward pass
 loss = output.loss
 loss.backward()
 optimizer.step()
print('Fine-tuning ended')


Fine-tuning started
Fine-tuning ended


In [ ]:


# input sentences
input_text = ["Andhra Pradesh may soon lose its position as the state with the second longest coastline in India to Tamil Nadu."]


# convert sentences to tensors
model_inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# translate from English to Tamil
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["ta_IN"]
)

translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

print(translation)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['தமிழ்நாட்டிடம் இந்தியாவின் இரண்டாவது நீண்ட கடலோர மாநிலம் ஆந்திரப் பிரதேசம் விரைவில் தனது இடத்தை இழந்துவிடும்.']
